## Reference

Most of this notebook is inspired from the wonderful gitrepos

1. [ml course ai hyperopt](https://github.com/Yorko/mlcourse.ai/blob/master/jupyter_english/tutorials/hyperparameters_tunning_ilya_larchenko.ipynb)

2. [Flaml github](https://github.com/microsoft/FLAML)

## Libraries import

In [1]:
# common imports
import os
import glob
import random
import numpy as np
import pandas as pd
from scipy.stats import randint

# models libraries
#from lightgbm.sklearn import LGBMRegressor
from lightgbm import LGBMRegressor,LGBMClassifier
from sklearn.svm import SVR

# sklearn imports 
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GroupKFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import make_pipeline

# hyperopt imports to perform bayesian optimisation 
from hyperopt import Trials, anneal, fmin, hp, tpe

%matplotlib inline

In [2]:
''' import AutoML class from flaml package '''
from flaml import AutoML
automl = AutoML()

## Helper functions

In [3]:
# the metric used in this competition
def comp_metric(xhat, yhat, fhat, x, y, f):
    intermediate = np.sqrt(np.power(xhat - x,2) + np.power(yhat-y,2)) + 15 * np.abs(fhat-f)
    return intermediate.sum()/xhat.shape[0]

def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    #torch.manual_seed(seed)
    #torch.cuda.manual_seed(seed)
    #torch.backends.cudnn.deterministic = True

SEED = 42
seed_everything(SEED)


# cv strategy 
N_FOLDS = 5
folds = GroupKFold(n_splits=N_FOLDS)

# which optimisation to perform
perform_RandomCVSearch = False
perform_hyperoptParsenEstimator = False
perform_hyperoptSimpleAnnealing = False
perfom_flaml = True


# number of experiments to perform for hyperopt
n_iter = 100

# target time for flaml, in seconds
timeLimit = 30  #~8 hours

## Read sample data

In [4]:
feature_dir = "referencePublicNotebooks/1000Features/"

# get our train and test files
train_files = sorted(glob.glob(os.path.join(feature_dir, 'train/*_train.csv')))
test_files = sorted(glob.glob(os.path.join(feature_dir, 'test/*_test.csv')))
ssubm = pd.read_csv('sample_submission.csv', index_col=0)
print(len(train_files),len(test_files))

24 24


In [5]:
# selecting a particular site and choosing y coorindate
e = 0
data = pd.read_csv(train_files[e], index_col=0)
print(data.shape)
data.head(3)

(9296, 945)


,000840e5c600de293cea57f13326f273c86c3988,00ad587dcb9c7ce3788b92e22777a22ee0efea31,00af060fc145ee6a6a50475efa57b91cbf54237f,00bcc61bdea4d52d050822d66952dd707c2fcdf3,00f0904087c01d922d6ebf3005607dfdeaf6687b,011e20ebf721a1c6dfec42e8ed1e2ac566073a2a,01d2f676abab6ec03ec5dc696bfd49d66e392ea1,01e25e4a25acd32baf5137b3031151f751fadbb4,026c2f057932da75680b21ecdbd23bf9cb9350f3,028a310e23177c3747d37971678dd964ee28ce17,...,fdc189e5a19850397f37201f4acc378cfddcf0d6,fdc19f011587b75c11a6c30d8ca06d90107b6bde,fdf37fa13679f581bdfaae3b99e368633e0a144b,fdfe926caf5f49a88a9bcab8d025e887f422128b,fe3211f90e4ab1f500e10fe175ae6142f4b13130,ffa41c79865d7fb336f586e0dec8b080db1027fb,x,y,f,path
4,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,...,-999,-999,-999,-999,-999,-999,155.65668,89.40598,-1,5e158edff4c3420006d52172
4,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,...,-999,-999,-999,-999,-999,-999,155.65668,89.40598,-1,5e158edff4c3420006d52172
4,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,...,-999,-999,-999,-999,-999,-999,154.68399,81.80792,-1,5e158edff4c3420006d52172


## Prepare model inputs and outputs

In [6]:
x_train = data.iloc[:,:-4].values.astype(int)
y_trainy = data.iloc[:,-3].values.astype(float)
y_trainx = data.iloc[:,-4].values.astype(float)
y_trainf = data.iloc[:,-2].values.astype(float)
groups = data["path"]

In [7]:
# normlise inputs
stdScaler = StandardScaler()
x_train = stdScaler.fit_transform(x_train)

## Baseline Lightgbm and SVR model 

In [8]:
%%time
# baseline lightgbm model
model = LGBMRegressor(n_estimators=125, num_leaves=90, random_state=SEED)
results = -cross_val_score(model, X=x_train, y=y_trainy, groups=groups, 
                              scoring="neg_mean_squared_error", cv=folds, n_jobs=-1)
print(f"Cross val score for y coordinate is {results.mean()}")
print(results)

Cross val score for y coordinate is 77.47947111520742
[90.20551365 92.8808168  53.35521555 68.3332076  82.62260198]
CPU times: user 55.1 ms, sys: 64.4 ms, total: 119 ms
Wall time: 17 s


In [9]:
# %%time
# baseline svm model
# svrModel = SVR(C=100.0, epsilon=0.01)
# results = -cross_val_score(svrModel, X=x_train, y=y_trainy, groups=groups, 
#                              scoring="neg_mean_squared_error", cv=folds, n_jobs=-1)
# print(f"Cross val score for y coordinate is {results.mean()}")
# print(results)

## Randomized grid search

In [10]:
%%time
if perform_RandomCVSearch == True:

    param_grid_rand = {
    "learning_rate": np.logspace(-5, 0, 100),
    "max_depth": randint(2, 20),
    "n_estimators": randint(100, 2000),
    "random_state": [SEED],
    }
    
    rs = RandomizedSearchCV(model,
        param_grid_rand,
        n_iter=n_iter,
        scoring="neg_mean_squared_error",
        #fit_params=None,
        n_jobs=-1,
        cv=folds,
        verbose=True,
        random_state=SEED,
    )

    rs.fit(x_train, y_trainy, groups=groups)
    print("Best MSE {:.3f} params {}".format(-rs.best_score_, rs.best_params_))

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.2 µs


In [11]:
if perform_RandomCVSearch == True:
    rs_results_df = pd.DataFrame(
        np.transpose(
            [
                -rs.cv_results_["mean_test_score"],
                rs.cv_results_["param_learning_rate"].data,
                rs.cv_results_["param_max_depth"].data,
                rs.cv_results_["param_n_estimators"].data,
            ]
        ),
        columns=["score", "learning_rate", "max_depth", "n_estimators"],
    )
    rs_results_df.plot(subplots=True, figsize=(10, 10))

## Hyperopt tuning methods
### Tree-structured Parzen Estimator and Simple Annealing

In [12]:
def gb_mse_cv(params, X=x_train, y=y_trainy, cv=folds,random_state=SEED):
    # the function gest a set of variable parameters in "param"
    lgb_params = {
        "objective": "regression",
        "metric": "l2",
        "verbosity": -1,
        
        # fixed params
        "boosting_type": "gbdt", 
        "subsample_freq":20,
        "max_depth":6,

        # variable parameters
        "num_leaves": int(params["num_leaves"]),
        "feature_fraction": float(params["feature_fraction"]),
        "bagging_fraction": float(params["bagging_fraction"]),        
        "learning_rate": float(params["learning_rate"]),
        "n_estimators": int(params["n_estimators"]),
        "lambda_l1": float(params["lambda_l1"]),
        "lambda_l2": float(params["lambda_l2"]),
        "min_child_samples": int(params["min_child_samples"]),
    }
    
    # we use this params to create a new LGBM Regressor
    model = LGBMRegressor(random_state=SEED, **lgb_params)

    # and then conduct the cross validation with the same folds as before
    score = -cross_val_score(model, X=X, y=y, groups=groups, scoring="neg_mean_squared_error",
                             cv=folds, n_jobs=-1).mean()
    return score

In [13]:
# possible values of parameters
space = {
        # variable parameters
        "num_leaves": hp.quniform("num_leaves", 10, 100, 1),
        "feature_fraction": hp.choice('feature_fraction', np.linspace(0.4, 0.7, 3,dtype=float)),
        "bagging_fraction": hp.choice('bagging_fraction', np.linspace(0.4, 0.7, 3,dtype=float)),        
        "learning_rate": hp.loguniform("learning_rate", -2, -1), 
        "n_estimators": hp.quniform("n_estimators", 500, 10000, 1),
        "lambda_l1": hp.loguniform("lambda_l1", -6, 1.0), 
        "lambda_l2": hp.loguniform("lambda_l2", -6, 1.0), 
        "min_child_samples": hp.quniform("min_child_samples", 5, 100, 1)
        }

# trials will contain logging information
trials = Trials()

In [15]:
tuningAlgorithm = None

# choice of tuning algorithm
if perform_hyperoptParsenEstimator == True:
    tuningAlgorithm = tpe.suggest
if perform_hyperoptSimpleAnnealing == True:
    tuningAlgorithm = anneal.suggest
if perfom_flaml == True:
    tuningAlgorithm = 'flaml'

In [16]:
print(tuningAlgorithm)

flaml


In [17]:
%%time
if((perform_hyperoptParsenEstimator == True) or (perform_hyperoptSimpleAnnealing == True)):
    best = fmin(
        fn=gb_mse_cv,                       # function to optimize
        space=space,                        # search space
        algo=tuningAlgorithm,               # optimization algorithm, hyperotp will select its parameters automatically
        max_evals=n_iter,                   # maximum number of iterations
        trials=trials,                      # logging
        show_progressbar=True,
        rstate=np.random.RandomState(SEED), # fixing random state for the reproducibility
    )
    print("Best MSE {:.3f} params {}".format(gb_mse_cv(best), best))

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 5.96 µs


## Plot optimizer results

In [18]:
if((perform_hyperoptParsenEstimator == True) or (perform_hyperoptSimpleAnnealing == True)):
    optimizer_results = np.array([[
                x["result"]["loss"],  
                x["misc"]["vals"]["n_estimators"][0],    
                x["misc"]["vals"]["learning_rate"][0],
                x["misc"]["vals"]["num_leaves"][0],
                x["misc"]["vals"]["feature_fraction"][0],
                x["misc"]["vals"]["bagging_fraction"][0],
                x["misc"]["vals"]["lambda_l1"][0],
                x["misc"]["vals"]["lambda_l2"][0],
                x["misc"]["vals"]["min_child_samples"][0],        
            ] for x in trials.trials ])

    # create a df to plot
    results_columns = ["score", "n_estimators", "learning_rate", "num_leaves", "feature_fraction",
                       "bagging_fraction", "lambda_l1", "lambda_l2", "min_child_samples"]
    optimizer_results_df = pd.DataFrame(optimizer_results, columns=results_columns)
    optimizer_results_df.plot(subplots=True, figsize=(10, 10));

In [19]:
if perfom_flaml == True:
    settings = {
        "metric": 'mse', # primary metrics for regression can be chosen from: ['mae','mse','r2']
        "task": 'regression', # task type        
        "log_file_name": 'lightgbm_ycoorindate.log', # flaml log file    
        "estimator_list": ['lgbm', 'xgboost'], # list of ML learners; we tune lightgbm in this example
        "time_budget": timeLimit, # total running time in seconds
        "eval_method": 'cv',
        "n_splits" : N_FOLDS, 
    }

    # fit algorithms
    automl.fit(X_train = x_train, y_train = y_trainy, **settings)

    print('Best hyperparmeter config:', automl.best_config)
    print('Best mse on validation data: {0:.4g}'.format(automl.best_loss))
    print('Training duration of best run: {0:.4g} s'.format(automl.best_config_train_time))
    
    print(automl.model)

[flaml.automl: 04-02 12:28:35] {884} INFO - Evaluation method: cv
[flaml.automl: 04-02 12:28:35] {601} INFO - Using RepeatedKFold
[flaml.automl: 04-02 12:28:35] {905} INFO - Minimizing error metric: mse
[flaml.automl: 04-02 12:28:35] {924} INFO - List of ML learners in AutoML Run: ['lgbm', 'xgboost']
[flaml.automl: 04-02 12:28:35] {986} INFO - iteration 0  current learner lgbm


[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] num_leaves is set=31, max_leaves=4 will be ignored. Current value: num_leaves=31
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] num_leaves is set=31, max_leaves=4 will be ignored. Current value: num_leaves=31
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] num_leaves is set=31, max_leaves=4 will be ignored. Current value: num_leaves=31
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] num_leaves is set=31, max_leaves=4 will be ignored. Current value: num_leaves=31
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: mi

[flaml.automl: 04-02 12:28:41] {1134} INFO -  at 6.2s,	best lgbm's error=822.7656,	best lgbm's error=822.7656
[flaml.automl: 04-02 12:28:41] {986} INFO - iteration 1  current learner lgbm


[LightGBM] [Warning] min_data_in_leaf is set=32, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=32
[LightGBM] [Warning] num_leaves is set=31, max_leaves=4 will be ignored. Current value: num_leaves=31
[LightGBM] [Warning] min_data_in_leaf is set=32, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=32
[LightGBM] [Warning] num_leaves is set=31, max_leaves=4 will be ignored. Current value: num_leaves=31
[LightGBM] [Warning] min_data_in_leaf is set=32, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=32
[LightGBM] [Warning] num_leaves is set=31, max_leaves=4 will be ignored. Current value: num_leaves=31
[LightGBM] [Warning] min_data_in_leaf is set=32, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=32
[LightGBM] [Warning] num_leaves is set=31, max_leaves=4 will be ignored. Current value: num_leaves=31


[flaml.automl: 04-02 12:28:45] {1134} INFO -  at 10.3s,	best lgbm's error=822.7656,	best lgbm's error=822.7656
[flaml.automl: 04-02 12:28:45] {986} INFO - iteration 2  current learner lgbm


...

[LightGBM] [Warning] min_data_in_leaf is set=8, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=8
[LightGBM] [Warning] num_leaves is set=31, max_leaves=7 will be ignored. Current value: num_leaves=31
[LightGBM] [Warning] min_data_in_leaf is set=8, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=8
[LightGBM] [Warning] num_leaves is set=31, max_leaves=7 will be ignored. Current value: num_leaves=31
[LightGBM] [Warning] min_data_in_leaf is set=8, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=8
[LightGBM] [Warning] num_leaves is set=31, max_leaves=7 will be ignored. Current value: num_leaves=31
[LightGBM] [Warning] min_data_in_leaf is set=8, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=8
[LightGBM] [Warning] num_leaves is set=31, max_leaves=7 will be ignored. Current value: num_leaves=31


[flaml.automl: 04-02 12:28:57] {1134} INFO -  at 22.1s,	best lgbm's error=56.2444,	best lgbm's error=56.2444
[flaml.automl: 04-02 12:28:57] {986} INFO - iteration 5  current learner xgboost
[flaml.automl: 04-02 12:28:58] {1134} INFO -  at 23.7s,	best xgboost's error=4887.5851,	best lgbm's error=56.2444
[flaml.automl: 04-02 12:28:58] {986} INFO - iteration 6  current learner lgbm


[LightGBM] [Warning] min_data_in_leaf is set=48, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=48
[LightGBM] [Warning] num_leaves is set=31, max_leaves=4 will be ignored. Current value: num_leaves=31
[LightGBM] [Warning] min_data_in_leaf is set=48, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=48
[LightGBM] [Warning] num_leaves is set=31, max_leaves=4 will be ignored. Current value: num_leaves=31
[LightGBM] [Warning] min_data_in_leaf is set=48, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=48
[LightGBM] [Warning] num_leaves is set=31, max_leaves=4 will be ignored. Current value: num_leaves=31
[LightGBM] [Warning] min_data_in_leaf is set=48, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=48
[LightGBM] [Warning] num_leaves is set=31, max_leaves=4 will be ignored. Current value: num_leaves=31
[LightGBM] [Warning] min_data_in_leaf is set=48, min_child_samples=20 will be ignored. Current value: mi

[flaml.automl: 04-02 12:29:05] {1134} INFO -  at 30.3s,	best lgbm's error=51.0423,	best lgbm's error=51.0423
[flaml.automl: 04-02 12:29:05] {1181} INFO - selected model: LGBMRegressor(colsample_bytree=0.9534346594834143, learning_rate=1.0,
              max_bin=1023, max_leaves=4, min_data_in_leaf=48, n_estimators=4,
              objective='regression', reg_alpha=0.0022085340760961856,
              reg_lambda=0.5460627024738886, subsample=0.9814787163243814)
[flaml.automl: 04-02 12:29:05] {939} INFO - fit succeeded


Best hyperparmeter config: {'n_estimators': 18.0, 'max_leaves': 4.0, 'min_data_in_leaf': 48.0, 'learning_rate': 1.0, 'subsample': 0.9814787163243814, 'log_max_bin': 10.0, 'colsample_bytree': 0.9534346594834143, 'reg_alpha': 0.0022085340760961856, 'reg_lambda': 0.5460627024738886}
Best mse on validation data: 51.04
Training duration of best run: 6.541 s
LGBMRegressor(colsample_bytree=0.9534346594834143, learning_rate=1.0,
              max_bin=1023, max_leaves=4, min_data_in_leaf=48, n_estimators=4,
              objective='regression', reg_alpha=0.0022085340760961856,
              reg_lambda=0.5460627024738886, subsample=0.9814787163243814)
